# Shift Cipher

Shift cipher is an encryption technique, where all characters of given text are shifted by a certain amount determined by a key, which is an integer value.

Assume a given text P needs to be encrypted using a key k, the cipher text C can be calculated as -
C = P + k

## Implementation

Firstly we define an encoding function that will be used to take plaintext and convert it to a 26 character encoding. (By converting all letters to upper case and discarding all remaining characters).

In [60]:
def encode(string):
    result = ''
    for letter in string:
        if letter.isalpha():
            result += letter.upper()
    return result

Lets declare a plain text that we would need to encrypt.

In [61]:
P = 'Enemy can attack tonight. Stay alert!'

Encoding this string, we get -

In [62]:
T = encode(P)
print(T)

ENEMYCANATTACKTONIGHTSTAYALERT


Since we will be only having 26 characters, we declare Zp as closed ring of 26 integers.

In [63]:
Zp = Integers(26)
print(Zp)

Ring of integers modulo 26


We now declare a key matrix for the encryption by choosing a key at random from the key domain.

In [64]:
from random import choice
k_domain = [i for i in range(26)] # integers from 0 to 25 (included)
key = choice(k_domain)

In [65]:
print('Key - ', key)

Key -  9


In [66]:
def shiftcipher(text, cipher_key):
    cipher = ''
    
    for e in text:
        text_value = ord(e) - ord('A')
        cipher_value = (text_value + cipher_key) % 26
        cipher += chr(int(cipher_value + ord('A')))
    return cipher
    

def shiftdecipher(cipher_text, cipher_key):
    text = ''
    for e in cipher_text:
        cipher_value = ord(e) - ord('A')
        text_value = (cipher_value - cipher_key) % 26
        text += chr(int(text_value + ord('A')))
    return text

Now we test the cipher and decipher algorithms by encrypting and decrypting the text P

In [67]:
T = encode(P)
C = shiftcipher(T, key)
D = shiftdecipher(C, key)
print(f'Given text - "{P}"')
print(f'Encoded - {T}')
print(f'Key - {key}')
print(f'Cipher text - {C}')
print(f'Decipher text - {D}')

Given text - "Enemy can attack tonight. Stay alert!"
Encoded - ENEMYCANATTACKTONIGHTSTAYALERT
Key - 9
Cipher text - NWNVHLJWJCCJLTCXWRPQCBCJHJUNAC
Decipher text - ENEMYCANATTACKTONIGHTSTAYALERT


## Test Against Builtin Cipher

Now, we can test the result against the built in Hill Cipher in sagemath.

In [68]:
A = ShiftCryptosystem(AlphabeticStrings())
E = A.encoding(encode(P))
print(f'Text - {P}')
print(f'Encoded - {E}')
print(f'Key -\b{key}')
C_test = A.enciphering(key, E)
D_test = A.deciphering(key, C_test)

# convert to python string
C_test = str(C_test)
D_test = str(D_test)

print(f'Cipher text - {C_test}')
print(f'Decipher text - {D_test}')

Text - Enemy can attack tonight. Stay alert!
Encoded - ENEMYCANATTACKTONIGHTSTAYALERT
Key -9
Cipher text - NWNVHLJWJCCJLTCXWRPQCBCJHJUNAC
Decipher text - ENEMYCANATTACKTONIGHTSTAYALERT


Comparing the built in cipher result with our implementation -

In [69]:
print('Results \t Implementation \t Built-in\n')
print(f'Cipher Text \t {C} \t {C_test}')
print(f'Decipher Text \t {D} \t {D_test}\n')
if C_test == C and D_test == D:
    print('Implementation is CORRECT')
else:
    print('Implementatiokn is INCORRECT')

Results 	 Implementation 	 Built-in

Cipher Text 	 NWNVHLJWJCCJLTCXWRPQCBCJHJUNAC 	 NWNVHLJWJCCJLTCXWRPQCBCJHJUNAC
Decipher Text 	 ENEMYCANATTACKTONIGHTSTAYALERT 	 ENEMYCANATTACKTONIGHTSTAYALERT

Implementation is CORRECT


## Cryptoanalysis

### Brute Force

Since the key domain for shift cipher, a brute force attack is very easy to carry out, making this cipher very weak.

In [70]:
print('Total keys -', len(k_domain))

Total keys - 26


We must now get a list of english words that can be used to detect existence of english words in our bruteforced decipher text.

A good list of 3000 most used english words is here -
https://github.com/aneeshsharma/EnglishWords/raw/main/common3000.txt

We download the list of words and convert it to a list and then convert all words into upper case.

In [71]:
import requests
url = 'https://github.com/aneeshsharma/EnglishWords/raw/main/common3000.txt'

words_file = requests.get(url, allow_redirects=True)
words_file_obj = open('words.txt', 'wb')
words_file_obj.write(words_file.content)
words_file_obj.close()

In [72]:
words = open('words.txt').read().split()
words = [word.upper() for word in words]

In [73]:
print(f'Number of words in dictionary - {len(words)}')

Number of words in dictionary - 3000


A function can be defined to find all substrings in a string that are among the 3000 most common english words. This can give us a measure of the likelihood of the string being an english sentance.

In [74]:
# function to find english words in a string according to word list
def find_words(string):
    l = len(string)
    found = []
    for i in range(l):
        for j in range(i, l):
            word = string[i:j+1]
            if len(word) <= 1:
                continue
            if word in words:
                found.append(string[i:j+1])
    return found

Now, we must try to decipher the encrypted text using the list of keys we have and try to compare and count any english words found in the text. More the words detected, more likely is it that the key is correct.

In [75]:
keys = {}
max_words = 0
for candidate in k_domain:
    candidate_text = shiftdecipher(C, candidate)
    found = find_words(candidate_text)
    if len(found) > 3:
        if len(found) > max_words:
            max_words = len(found)
        keys[candidate] = len(found)

print('Key \t\t Likelihood')
for likely_key in keys:
    print(f'{likely_key} \t\t {keys[likely_key]}')

Key 		 Likelihood
2 		 4
9 		 10
20 		 4


Now that we have a list of keys and their likelihood of being correct, we can display the keys and the possible plain text that are the most likely to be correct.

In [76]:
text_list = [[] for _ in range(max_words + 1)]
for likely_key in keys:
    count = keys[likely_key]
    text_list[count].append(shiftdecipher(C, likely_key))

print('Most likely strings -')
for text in text_list[max_words]:
    print(f'{text}')

Most likely strings -
ENEMYCANATTACKTONIGHTSTAYALERT


### Statistical Mono